In [1]:
import sys
import os
# navigate two directories up
dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(dir)
# print working directory
# mavigate to the top of working directory
os.chdir('../../')
os.chdir('statemanager')
print(os.getcwd())
from vspace.vsearch import VectorSearch
import pickle

/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/repomanager/statemanager


/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pwd

'/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/repomanager/statemanager'

In [3]:
sys.path

['/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/repomanager/tests/documentation_vector_search',
 '/Users/chinmayshrivastava/anaconda3/lib/python311.zip',
 '/Users/chinmayshrivastava/anaconda3/lib/python3.11',
 '/Users/chinmayshrivastava/anaconda3/lib/python3.11/lib-dynload',
 '',
 '/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/venv/lib/python3.11/site-packages',
 '/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager']

In [4]:
repo_id = "45526e5b-f544-4016-8381-f88f5ca095ea"

In [5]:
G = pickle.load(open("state/{}/state_0.pkl".format(repo_id), "rb"))
print(G)

DiGraph with 31986 nodes and 53803 edges


In [8]:
vs = VectorSearch(
    collection_name="docs",
    collection_path="state/{}/meta/storage".format(repo_id),
)

In [9]:
vs.collection.count()

14163

In [10]:
query = """For the user query, match the best use case or insight from a code base.
user query: How to rerank"""
nodes = [x["node"] for x in vs.search_all(query)['metadatas'][0]]
distances = vs.search_all(query)['distances'][0]
distances

[0.16169559955596924,
 0.16963738203048706,
 0.17061495780944824,
 0.17151081562042236,
 0.17151081562042236]

In [11]:
i = 0
for node in nodes:
    # get the node from the graph and get the content
    print(G.nodes[node]["content"])
    # print(G.nodes[node[:len(node)-1]+f"{int(node[-1])-1}"]["content"])
    i = i + 1
    if i == 3:
        break

Define Advanced Retriever
We define an advanced retriever that performs the following steps:

Query generation/rewriting: generate multiple queries given the original user query
Perform retrieval for each query over an ensemble of retrievers.
Reranking/fusion: fuse results from all queries, and apply a reranking step to "fuse" the top relevant results!

Then in the next section we'll plug this into our response synthesis module.

Step 1: Query Generation/Rewriting
The first step is to generate queries from the original query to better match the query intent, and increase precision/recall of the retrieved results. For instance, we might be able to rewrite the query into smaller queries.
We can do this by prompting ChatGPT.

input:
```python
from llama_index import PromptTemplate
```
output:
input:
```python
query_str = "How do the models developed in this work compare to open-source chat models based on the benchmarks tested?"
```
output:
input:
```python
query_gen_prompt_str = (
    "Y

In [104]:
from agents.llama_agents.FunctionCaller import LLMReranker
from defaults.vsearch import DISTANCE_THRESHOLD_cdb
def get_docs(
    query: str,
    vs: VectorSearch,
    G,
    top_k: int = 25,
    max_depth: int = 3,
    to_return: int = 2,
    ):
    res = vs.search_all(query, top_k=top_k)
    nodes = [x["node"] for x in res['metadatas'][0]]
    distances = res['distances'][0]
    docs = []
    for node in nodes:
        docs.append(G.nodes[node]["content"])
    i = 0
    _new_docs = []
    for doc, dist in zip(docs, distances):
        if dist > DISTANCE_THRESHOLD_cdb:
            continue
        if i>=5:
            break
        if len(doc.split()) < 10:
            continue
        if len(doc.split()) > 200:
            doc = " ".join(doc.split()[:200])
        _new_docs.append(doc)
        i = i + 1
    reranker = LLMReranker()
    reranked_docs = reranker.rerank(query, _new_docs)
    print(reranked_docs)
    docs = reranked_docs[:to_return]
    return docs, res


In [105]:
query = """For the user query, match the best use case or insight from a code base.
user query: What are the different types of agents?"""
a, b = get_docs(query, vs, G)
l = sum([len(x.split()) for x in a])
print(l)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


5 3
[3, 1, 2]
["\n\nChat Engine - ReAct Agent Mode\n\nReAct is an agent based chat mode built on top of a query engine over your data.\n\nFor each chat interaction, the agent enter a ReAct loop:\n\nfirst decide whether to use the query engine tool and come up with appropriate input\n(optional) use the query engine tool and observe its output\ndecide whether to repeat or give final response\n\n\nThis approach is flexible, since it can flexibility choose between querying the knowledge base or not.\nHowever, the performance is also more dependent on the quality of the LLM.\nYou might need to do more coercing to make sure it chooses to query the knowledge base at right times, instead of hallucinating an answer.\n\nIf you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.\n\ninput:\n```python\n!pip install llama-index\n```\noutput:\n", 'Run Some Example Queries\nWe run some example queries using the agent, showcasing some of the agent\'s abilities to do chain

In [45]:
a

["\n\nChat Engine - ReAct Agent Mode\n\nReAct is an agent based chat mode built on top of a query engine over your data.\n\nFor each chat interaction, the agent enter a ReAct loop:\n\nfirst decide whether to use the query engine tool and come up with appropriate input\n(optional) use the query engine tool and observe its output\ndecide whether to repeat or give final response\n\n\nThis approach is flexible, since it can flexibility choose between querying the knowledge base or not.\nHowever, the performance is also more dependent on the quality of the LLM.\nYou might need to do more coercing to make sure it chooses to query the knowledge base at right times, instead of hallucinating an answer.\n\nIf you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.\n\ninput:\n```python\n!pip install llama-index\n```\noutput:\n"]

In [30]:
b

{'ids': [['multi_document_agents-v1.ipynb - 5 - 9',
   'react_agent_with_query_engine.ipynb - 3 - 1',
   'chat_engine_react.ipynb - 0 - 0',
   'openai_assistant_query_cookbook.ipynb - 4 - 2',
   'openai_agent_query_plan.ipynb - 4 - 8']],
 'distances': [[0.14939218759536743,
   0.15069550275802612,
   0.15483319759368896,
   0.15509891510009766,
   0.15511804819107056]],
 'metadatas': [[{'node': 'multi_document_agents-v1.ipynb - 5',
    'type': 'USE_CASE'},
   {'node': 'react_agent_with_query_engine.ipynb - 3', 'type': 'INSIGHT'},
   {'node': 'chat_engine_react.ipynb - 0', 'type': 'INSIGHT'},
   {'node': 'openai_assistant_query_cookbook.ipynb - 4', 'type': 'INSIGHT'},
   {'node': 'openai_agent_query_plan.ipynb - 4', 'type': 'USE_CASE'}]],
 'embeddings': None,
 'documents': [['how to build a top-level agent that can orchestrate across different document agents to answer user queries',
   'the agent can provide analysis based on the given query',
   'The agent can decide whether to use th

In [31]:
# rerank

import nltk

# tokinize
def tokenize(text):
    return nltk.word_tokenize(text)

# rank the documents according to the tokenized query
def rank_docs(docs, query):
    query_tokens = tokenize(query)
    scores = []
    for doc in docs:
        doc_tokens = tokenize(doc)
        score = 0
        for token in query_tokens:
            score += doc_tokens.count(token)
        scores.append(score)
    reranked_docs = [x for _,x in sorted(zip(scores,docs), reverse=True)]
    return reranked_docs

In [32]:
query = """Find the best matching usecase or insight.
user query: What is fusion retrieval?"""

docs = []
for doc in get_docs(query, vs, G, top_k=5, max_depth=3):
    docs.extend(doc)
docs

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


3 2
[1, 2]


['Simple Fusion Retriever\nIn this example, we walk through how you can combine retrieval results from multiple queries and multiple indexes.\nThe retrieved nodes will be returned as the top-k across all queries and indexes, as well as handling de-duplication of any nodes.\n\ninput:\n```python\nimport os\nimport openai\n\nos.environ["OPENAI_API_KEY"] = "sk-..."\nopenai.api_key = os.environ["OPENAI_API_KEY"]\n```\noutput:\n',
 'Create a Hybrid Fusion Retriever\nIn this step, we fuse our index with a BM25 based retriever. This will enable us to capture both semantic relations and keywords in our input queries.\nSince both of these retrievers calculate a score, we can use the reciprocal rerank algorithm to re-sort our nodes without using an additional models or excessive computation.\nThis setup will also query 4 times, once with your original query, and generate 3 more queries.\nBy default, it uses the following prompt to generate extra queries:\nQUERY_GEN_PROMPT = (\n    "You are a help

In [33]:
rank_docs(docs, query)

['Create a Hybrid Fusion Retriever\nIn this step, we fuse our index with a BM25 based retriever. This will enable us to capture both semantic relations and keywords in our input queries.\nSince both of these retrievers calculate a score, we can use the reciprocal rerank algorithm to re-sort our nodes without using an additional models or excessive computation.\nThis setup will also query 4 times, once with your original query, and generate 3 more queries.\nBy default, it uses the following prompt to generate extra queries:\nQUERY_GEN_PROMPT = (\n    "You are a helpful assistant that generates multiple search queries based on a "\n    "single input query. Generate {num_queries} search queries, one on each line, "\n    "related to the following input query:\\n"\n    "Query: {query}\\n"\n    "Queries:\\n"\n)\n\n\nFirst, we create our retrievers. Each will retrieve the top-2 most similar nodes:\n\ninput:\n```python\nfrom llama_index.retrievers import BM25Retriever\n\nvector_retriever = ind

In [34]:
from repomanager.statemanager.llms.completion import get_llm

llm = get_llm()
llm2 = get_llm(model='gpt-4')

In [138]:
query = 'What are fusion techniques?'
_query = """For the user query, match the best use case or insight from a code base.
user query: {}""".format(query)
# docs = []
# for doc in get_docs(_query, vs, G, top_k=5, max_depth=3)[0]:
#     docs.extend(doc)
# reranked_docs = rank_docs(docs, _query)
# top_three = reranked_docs[:3]
docs = get_docs(_query, vs, G, to_return=2)[0]
print("docs: ", docs)
print(len(docs))
prompt = """For the user query, use the information attached to answer it in short form.
Provide simple examples (if possible):\n"""
i = 0
for doc in docs:
    prompt += "Source {}\n".format(i)
    prompt += doc + "\n"
    i += 1
prompt += "Query:\n"
prompt += query + "\n"
prompt += "Answer:\n"
# print the number of words in the prompt
prompt

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


5 3
[2, 3, 5]
['Define Advanced Retriever We define an advanced retriever that performs the following steps: Query generation/rewriting: generate multiple queries given the original user query Perform retrieval for each query over an ensemble of retrievers. Reranking/fusion: fuse results from all queries, and apply a reranking step to "fuse" the top relevant results! Then in the next section we\'ll plug this into our response synthesis module. Step 1: Query Generation/Rewriting The first step is to generate queries from the original query to better match the query intent, and increase precision/recall of the retrieved results. For instance, we might be able to rewrite the query into smaller queries. We can do this by prompting ChatGPT. input: ```python from llama_index import PromptTemplate ``` output: input: ```python query_str = "How do the models developed in this work compare to open-source chat models based on the benchmarks tested?" ``` output: input: ```python query_gen_prompt_s

'For the user query, use the information attached to answer it in short form.\nProvide simple examples (if possible):\nSource 0\nDefine Advanced Retriever We define an advanced retriever that performs the following steps: Query generation/rewriting: generate multiple queries given the original user query Perform retrieval for each query over an ensemble of retrievers. Reranking/fusion: fuse results from all queries, and apply a reranking step to "fuse" the top relevant results! Then in the next section we\'ll plug this into our response synthesis module. Step 1: Query Generation/Rewriting The first step is to generate queries from the original query to better match the query intent, and increase precision/recall of the retrieved results. For instance, we might be able to rewrite the query into smaller queries. We can do this by prompting ChatGPT. input: ```python from llama_index import PromptTemplate ``` output: input: ```python query_str = "How do the models developed in this work co

In [139]:
display(Markdown(f'{docs}'))

['Define Advanced Retriever We define an advanced retriever that performs the following steps: Query generation/rewriting: generate multiple queries given the original user query Perform retrieval for each query over an ensemble of retrievers. Reranking/fusion: fuse results from all queries, and apply a reranking step to "fuse" the top relevant results! Then in the next section we\'ll plug this into our response synthesis module. Step 1: Query Generation/Rewriting The first step is to generate queries from the original query to better match the query intent, and increase precision/recall of the retrieved results. For instance, we might be able to rewrite the query into smaller queries. We can do this by prompting ChatGPT. input: ```python from llama_index import PromptTemplate ``` output: input: ```python query_str = "How do the models developed in this work compare to open-source chat models based on the benchmarks tested?" ``` output: input: ```python query_gen_prompt_str = ( "You are a helpful assistant that generates multiple search queries based on a " "single input query. Generate {num_queries} search queries, one on each line, " "related to the following input query:\\n" "Query: {query}\\n" "Queries:\\n" ) query_gen_prompt = PromptTemplate(query_gen_prompt_str) ``` output: input: ```python def generate_queries(llm, query_str: str, num_queries: int = 4): fmt_prompt', 'Advanced RAG\nThe GoogleIndex is built based on GoogleVectorStore and GoogleTextSynthesizer.\nThese components can be combined with other powerful constructs in LlamaIndex to produce advanced RAG applications.\nBelow we show a few examples.\n\nReranker + Google Retriever\nConverting content into vectors is a lossy process. LLM-based Reranking\nremediates this by reranking the retrieved content using LLM, which has higher\nfidelity because it has access to both the actual query and the passage.\n\ninput:\n```python\nfrom llama_index.response_synthesizers.google.generativeai import (\n    GoogleTextSynthesizer,\n)\nfrom llama_index.vector_stores.google.generativeai import (\n    GoogleVectorStore,\n    google_service_context,\n)\nfrom llama_index import ServiceContext, VectorStoreIndex\nfrom llama_index.llms import PaLM\nfrom llama_index.postprocessor import LLMRerank\nfrom llama_index.query_engine import RetrieverQueryEngine\nfrom llama_index.retrievers import VectorIndexRetriever\n\n# Set up the query engine with a reranker.\nstore = GoogleVectorStore.from_corpus(corpus_id="some-corpus-id")\nindex = VectorStoreIndex.from_vector_store(\n    vector_store=store, service_context=google_service_context\n)\nresponse_synthesizer = GoogleTextSynthesizer.from_defaults(\n    temperature=0.7, answer_style=GenerateAnswerRequest.AnswerStyle.ABSTRACTIVE\n)\nreranker = LLMRerank(\n    top_n=10, service_context=ServiceContext.from_defaults(llm=PaLM())\n)\nquery_engine = RetrieverQueryEngine.from_args(\n    retriever=VectorIndexRetriever(\n        index=index,\n        similarity_top_k=20,\n    ),\n    response_synthesizer=response_synthesizer,\n    node_postprocessors=[reranker],\n)\n\n# Query for better result!\nresponse = query_engine.query("What movie should I watch with my family?")\n```\noutput:\n']

In [140]:
# import display and markdown
from IPython.display import display, Markdown

display(Markdown(llm.complete(prompt).text))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Fusion techniques are methods used to combine or merge results from multiple queries or retrievers to generate a final set of relevant results. This can be done through various approaches such as ranking, weighting, or combining the results based on their relevance scores. The goal is to improve the overall quality and accuracy of the retrieved information.

In [141]:
display(Markdown(llm2.complete(prompt).text))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Fusion techniques refer to the methods used to combine or "fuse" results from multiple queries in an advanced retriever. After generating multiple queries from the original user query and retrieving results for each, the advanced retriever uses fusion techniques to combine these results. This is followed by a reranking step to prioritize the most relevant results. For example, if the original query is "What movie should I watch with my family?", the retriever might generate related queries like "Family-friendly movies", "Popular movies for families", etc., retrieve results for each, and then fuse and rerank these results to provide the best answer.

In [157]:
# perform page rank on the graph
import networkx as nx
import numpy as np

page_rank = nx.pagerank(G)
page_rank

{'root': 2.535797987573841e-05,
 'LICENSE': 2.6558362187526002e-05,
 'CHANGELOG.md': 2.6558362187526002e-05,
 '.pre-commit-config.yaml': 2.6558362187526002e-05,
 'Makefile': 2.8612681822994494e-05,
 'CITATION.cff': 2.6558362187526002e-05,
 'pyproject.toml': 2.6558362187526002e-05,
 'README.md': 9.657432073854376e-05,
 'CONTRIBUTING.md': 2.6558362187526002e-05,
 'poetry.lock': 2.6558362187526002e-05,
 '.readthedocs.yaml': 2.6558362187526002e-05,
 'experimental': 2.7223581194882335e-05,
 'splitter_playground': 2.8673215955606596e-05,
 'app.py': 4.9838092699503767e-05,
 'cli': 2.8673215955606596e-05,
 'configuration.py': 3.858561624856383e-05,
 'cli_add.py': 3.0910707139595646e-05,
 '__init__.py': 0.0006499568398591388,
 'cli_init.py': 3.0910707139595646e-05,
 'cli_query.py': 3.0910707139595646e-05,
 '__main__.py': 2.885513885056203e-05,
 'openai_fine_tuning': 2.8673215955606596e-05,
 'openai_fine_tuning.ipynb': 3.586639057649473e-05,
 'launch_training.py': 3.351801748366019e-05,
 'valida

In [162]:
page_rank['OpenAIAgent'], page_rank['AgentRunner'], page_rank["BaseAgentRunner"], page_rank["BaseReader"], page_rank["BaseAgent"]

(4.1214549799908395e-05,
 3.139584137403907e-05,
 2.818730587641715e-05,
 0.00021939831410859982,
 3.34182345009083e-05)

In [164]:
vs2 = VectorSearch(
    collection_name="explanations",
    collection_path="repomanager/statemanager/state/{}/meta/storage".format(repo_id),
)

In [186]:
query = "How to load a pdf"
res = vs2.search(query, top_k=5, type="class")
res

[('PDFReader', 'class'),
 ('DocxReader', 'class'),
 ('BaseReader', 'class'),
 ('GoogleDocsReader', 'class'),
 ('MyScaleReader', 'class')]

In [187]:
# rank res by page rank
res_ranked = sorted(res, key=lambda x: page_rank[x[0]], reverse=True)
res_ranked

[('BaseReader', 'class'),
 ('PDFReader', 'class'),
 ('GoogleDocsReader', 'class'),
 ('DocxReader', 'class'),
 ('MyScaleReader', 'class')]